<a href="https://colab.research.google.com/github/DaniellePotts/Sentiment-Analysis-with-QReLU-and-mQReLU/blob/main/ReLU_sentiments_140.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install contractions
!python -m spacy download en

     |████████████████████████████████| 321 kB 7.6 MB/s 
     |████████████████████████████████| 284 kB 61.7 MB/s 
  Created wheel for pyahocorasick: filename=pyahocorasick-1.4.2-cp37-cp37m-linux_x86_64.whl size=85447 sha256=a6afa504cd80e836a8e4a4f80bf06dc034b7fd4623caa8d5b0905786bc0a1101
  Stored in directory: /root/.cache/pip/wheels/25/19/a6/8f363d9939162782bb8439d886469756271abc01f76fbd790f
Successfully built pyahocorasick
     |████████████████████████████████| 12.0 MB 7.5 MB/s 
✔ Download and installation successful
You can now load the model via spacy.load('en_core_web_sm')
✔ Linking successful
/usr/local/lib/python3.7/dist-packages/en_core_web_sm -->
/usr/local/lib/python3.7/dist-packages/spacy/data/en
You can now load the model via spacy.load('en')


In [4]:
%tensorflow_version 1.x 
import tensorflow as tf
sess = tf.Session(config=tf.ConfigProto(log_device_placement=True))
import pandas as pd  
import numpy as np
import matplotlib.pyplot as plt
from bs4 import BeautifulSoup
import spacy
import re
import unicodedata
import nltk
nltk.download('stopwords')
import contractions
from collections import Counter
from nltk.tokenize.toktok import ToktokTokenizer

from sklearn.model_selection import train_test_split
from keras.preprocessing import sequence

from QReLU_TensorFlow_Keras import QReLU

Device mapping:
/job:localhost/replica:0/task:0/device:XLA_CPU:0 -> device: XLA_CPU device

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [5]:
nlp = spacy.load('en', parse=True, tag=True, entity=True)
tokenizer = ToktokTokenizer()
stopword_list = nltk.corpus.stopwords.words('english')
stopword_list.remove('no')
stopword_list.remove('not')

In [6]:
cols = ['sentiment','id','date','query_string','user','text']

In [7]:
df = pd.read_csv("./drive/MyDrive/QReLU/training.csv",header=None, names=cols,  engine='python')

In [8]:
sentiments_cat = ['negative','neutral','postive']

In [9]:
def chop_dataframe(df, amount):
  df_4 = df.loc[df['sentiment'] == 4]
  df_1 = df.loc[df['sentiment'] == 0]
  df_final = df_4.head(amount)
  return pd.concat([df_final, df_1.head(amount)])

def encode_corpus(corpus):
  words = ' '.join(corpus).split()
  
  count_words = Counter(words)
  total_words = len(words)

def count_words(corpus):
  words = ' '.join(corpus).split()
  
  counted_words = Counter(words)
  total_words = len(words)

  return total_words, counted_words

def sort_words(corpus):
  total_words, counted_words = count_words(corpus)
  return counted_words.most_common(total_words)

def decode_sentiments(encoded_sentiments, sentiments_cat=sentiments_cat, postive_index=2, 
                      negative_index=0, neutral_index=1):
  decoded_sentiments = []

  for index in range(0, len(encoded_sentiments)):
    if encoded_sentiments[index] == 0:
      decoded_sentiments.append(sentiments_cat[negative_index])
    elif encoded_sentiments[index] == 2:
      decoded_sentiments.append(sentiments_cat[neutral_index])
    elif encoded_sentiments[index] == 4 or encoded_sentiments[index] == 1:
      decoded_sentiments.append(sentiments_cat[postive_index])
  
  return decoded_sentiments

def convert_sentiments(sentiments):
  converted = []

  for s in sentiments:
    if s == 4:
      converted.append(1)
    else:
      converted.append(0)
  return np.array(converted)

In [10]:
#save load json files
import json 

def save(file_name, obj):
  with open(file_name, 'w') as outfile:
    json.dump(obj, outfile)

def load(file_name):
  with open(file_name) as json_file:
    data = json.load(json_file)
    return data['corpus']

In [11]:
def strip_html_tags(text):
    soup = BeautifulSoup(text, "html.parser")
    stripped_text = soup.get_text()
    return stripped_text

def convert_to_lower(corpus):
    return [c.lower() for c in corpus]
    
def remove_accented_chars(text):
    text = unicodedata.normalize('NFKD', text).encode('ascii', 'ignore').decode('utf-8', 'ignore')
    return text

def remove_special_characters(text, remove_digits=False):
    pattern = r'[^a-zA-z0-9\s]' if not remove_digits else r'[^a-zA-z\s]'
    text = re.sub(pattern, '', text)
    return text

def remove_stopwords(text, is_lower_case=False):
    tokens = tokenizer.tokenize(text)
    tokens = [token.strip() for token in tokens]
    if is_lower_case:
        filtered_tokens = [token for token in tokens if token not in stopword_list]
    else:
        filtered_tokens = [token for token in tokens if token.lower() not in stopword_list]
    filtered_text = ' '.join(filtered_tokens)    
    return filtered_text

def expand_contractions(text):
  expanded_words = []    
  for word in text.split():
    # using contractions.fix to expand the shotened words
    expanded_words.append(contractions.fix(word))   
      
  return' '.join(expanded_words)

In [12]:
df_4 = df.loc[df['sentiment'] == 4]
df_1 = df.loc[df['sentiment'] == 0]
df_final = df_4.head(60000)
df_5 = pd.concat([df_final, df_1.head(60000)])

In [13]:
df = df_5

In [14]:
len(df_5)

120000

In [15]:
sentiments = convert_sentiments(df['sentiment'].values)

In [16]:
sentiments = convert_sentiments(df['sentiment'].values)
corpus = convert_to_lower(df['text'].values)

In [17]:
def normalize_corpus(corpus, html_stripping=True, contraction_expansion=True,
                     accented_char_removal=True, text_lower_case=True, 
                     text_lemmatization=True, special_char_removal=True, 
                     stopword_removal=True, remove_digits=True):
    
    normalized_corpus = []
    # normalize each document in the corpus
    for doc in corpus:
        # strip HTML
        if html_stripping:
            doc = strip_html_tags(doc)
        
        # remove accented characters
        if accented_char_removal:
            doc = remove_accented_chars(doc)
        
        # expand contractions    
        if contraction_expansion:
            doc = expand_contractions(doc)
        
        # lowercase the text    
        if text_lower_case:
            doc = doc.lower()
        
        # remove extra newlines
        doc = re.sub(r'[\r|\n|\r\n]+', ' ',doc)
        
        # lemmatize text
        # if text_lemmatization:
        #     doc = lemmatize_text(doc)
        
        # remove special characters and\or digits    
        if special_char_removal:
            # insert spaces between special characters to isolate them    
            special_char_pattern = re.compile(r'([{.(-)!}])')
            doc = special_char_pattern.sub(" \\1 ", doc)
            doc = remove_special_characters(doc, remove_digits=remove_digits)  
            
            pat1 = r'@[A-Za-z0-9_]+'
            pat2 = r'https?://[^ ]+'
            www_pat = r'www.[^ ]+'
            combined_pat = r'|'.join((pat1, pat2))
            doc = re.sub(combined_pat, '', doc)
            doc = re.sub(www_pat, '', doc)

        # remove extra whitespace
        doc = re.sub(' +', ' ', doc)
        # remove stopwords
        if stopword_removal:
            doc = remove_stopwords(doc, is_lower_case=text_lower_case)
            
        normalized_corpus.append(doc)
        
    return normalized_corpus

In [18]:
normalized_corpus = normalize_corpus(corpus)
save("./drive/MyDrive/QReLU/corpus.json", {"corpus": normalized_corpus})

# normalized_corpus = load("./drive/MyDrive/QReLU/corpus.json")

In [ ]:
normalized_corpus[0]

'love healthuandpets guys r best'

In [19]:
total_words = count_words(normalized_corpus)
sorted_words = sort_words(normalized_corpus)

In [20]:
vocab_to_int = {w:i+1 for i, (w,c) in enumerate(sorted_words)}

In [21]:
def tokenize(corpus):
  reviews_int = []
  for review in corpus:
      r = [vocab_to_int[w] for w in review.split()]
      reviews_int.append(r)
  return reviews_int

In [22]:
encoded = tokenize(normalized_corpus)
encoded_np = np.array(encoded)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  


In [ ]:
len(encoded)

20000

In [23]:
VOCAB_SIZE = total_words[0]
MAXLEN = 160

X_train, X_test, y_train, y_test = train_test_split(
    encoded, sentiments, test_size=0.2, random_state=42)

X_train = sequence.pad_sequences(X_train, MAXLEN)
X_test = sequence.pad_sequences(X_test, MAXLEN)

In [25]:
model2 = tf.keras.Sequential([
    tf.keras.layers.Embedding(VOCAB_SIZE, 128),
    tf.keras.layers.LSTM(128, return_sequences=True),
    tf.keras.layers.LSTM(128),
    tf.keras.layers.Dense(1, activation="relu")
])

model2.summary()

Instructions for updating:
If using Keras pass *_constraint arguments to layers.
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, None, 128)         116228352 
_________________________________________________________________
lstm_2 (LSTM)                (None, None, 128)         131584    
_________________________________________________________________
lstm_3 (LSTM)                (None, 128)               131584    
_________________________________________________________________
dense (Dense)                (None, 1)                 129       
Total params: 116,491,649
Trainable params: 116,491,649
Non-trainable params: 0
_________________________________________________________________


In [26]:
model2.compile(loss="binary_crossentropy",optimizer="RMSprop",metrics=['acc'])

history = model2.fit(X_train, y_train, epochs=5, validation_split=0.3, batch_size=32)

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Train on 67200 samples, validate on 28800 samples
Epoch 1/5
67200/67200 [==============================] - 2866s 43ms/sample - loss: 0.5906 - acc: 0.7324 - val_loss: 0.5173 - val_acc: 0.7563
Epoch 2/5
67200/67200 [==============================] - 2864s 43ms/sample - loss: 0.5194 - acc: 0.7916 - val_loss: 0.6233 - val_acc: 0.7722
Epoch 3/5
67200/67200 [==============================] - 2865s 43ms/sample - loss: 0.4713 - acc: 0.8303 - val_loss: 0.7390 - val_acc: 0.7707
Epoch 4/5
67200/67200 [==============================] - 2848s 42ms/sample - loss: 0.4452 - acc: 0.8795 - val_loss: 0.7288 - val_acc: 0.7432
Epoch 5/5
67200/67200 [==============================] - 2835s 42ms/sample - loss: 0.3979 - acc: 0.9063 - val_loss: 1.1552 - val_acc: 0.7516


In [27]:
results = model2.evaluate(X_test, y_test)
print(results)

24000/24000 [==============================] - 71s 3ms/sample - loss: 1.1636 - acc: 0.7477
[1.1636184007326762, 0.7477083]
